# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [3]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 12.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630217 sha256=e4415da4fe07b114587158fc9cfcb99ac2bdec0d6c5a232fb4b1d4de866798f7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

간단한 surprise 실습

In [ ]:
data=Dataset.load_buil

In [5]:
data=Dataset.load_builtin('ml-100k',prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [6]:
model=SVD()

In [9]:
cross_validate(model,data,measures=['rmse','mae'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9386  0.9303  0.9373  0.9340  0.9386  0.9357  0.0032  
MAE (testset)     0.7409  0.7325  0.7401  0.7331  0.7396  0.7373  0.0036  
Fit time          5.18    5.12    4.85    4.78    4.72    4.93    0.18    
Test time         0.20    0.27    0.17    0.18    0.17    0.20    0.04    


{'fit_time': (5.1766698360443115,
  5.116447925567627,
  4.849559545516968,
  4.780054807662964,
  4.720704793930054),
 'test_mae': array([0.74087576, 0.73251391, 0.74012069, 0.73313616, 0.73964971]),
 'test_rmse': array([0.93861055, 0.93027393, 0.93725261, 0.93401596, 0.93858507]),
 'test_time': (0.195265531539917,
  0.2708101272583008,
  0.17180299758911133,
  0.1752781867980957,
  0.1747267246246338)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [10]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [11]:
data=Dataset.load_builtin('ml-100k',prompt=False)
raw_data=np.array(data.raw_ratings,dtype=int)

In [12]:
raw_data[:,0]-=1
raw_data[:,1]-=1

In [14]:
n_users=np.max(raw_data[:,0])
n_movies=np.max(raw_data[:,1])
shape=(n_users+1,n_movies+1)
shape

(943, 1682)

In [15]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=1.
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [16]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    similarity=np.dot(my_vector,user_vector)
    if similarity > best_match:
      best_match=similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 183, Best Match ID : 275


In [19]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [24]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=9999,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist=np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match=euclidean_dist
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 14.832396974191326, Best Match ID : 737


In [25]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [28]:
def compute_cos_similarity(v1,v2):
  norm1=np.sqrt(np.sum(np.square(v1)))
  norm2=np.sqrt(np.sum(np.square(v2)))
  dot=np.dot(v1,v2)
  return dot/(norm1*norm2)

In [30]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.5278586163659506, Best Match ID : 915


915 qjsdlqhkTwlaks sksms dksqhs dudghkemfdmf cncjsgownsek.

In [31]:
recommend_list=[]
for i, log in enumerate(zip(my_vector,best_match_vector)):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [33]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [34]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=9999,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist=np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match=euclidean_dist
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 55.06359959174482, Best Match ID : 737


In [35]:
my_id,my_vector=0,adj_matrix[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.569065731527988, Best Match ID : 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [37]:
from surprise import KNNBasic, SVD, SVDpp,NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [38]:
data=Dataset.load_builtin('ml-100k',prompt=False)

* KNN을 사용한 협업 필터링

In [39]:
model=KNNBasic()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9810  0.9796  0.9765  0.9792  0.9720  0.9776  0.0032  
MAE (testset)     0.7737  0.7746  0.7709  0.7734  0.7690  0.7723  0.0021  
Fit time          0.32    0.54    0.46    0.43    0.45    0.44    0.07    
Test time         7.49    8.20    8.15    7.20    3.82    6.97    1.62    


{'fit_time': (0.32428431510925293,
  0.537247896194458,
  0.45870494842529297,
  0.428896427154541,
  0.44632720947265625),
 'test_mae': array([0.77373531, 0.77456674, 0.77088424, 0.77339578, 0.76899244]),
 'test_rmse': array([0.98096715, 0.97955353, 0.97647973, 0.9792246 , 0.9720201 ]),
 'test_time': (7.4903788566589355,
  8.20331621170044,
  8.152160167694092,
  7.199713230133057,
  3.824502468109131)}

* SVD를 사용한 협업 필터링

In [40]:
model=SVD()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9372  0.9290  0.9390  0.9374  0.9416  0.9368  0.0042  
MAE (testset)     0.7374  0.7301  0.7435  0.7358  0.7412  0.7376  0.0046  
Fit time          10.97   14.04   14.00   11.79   7.78    11.72   2.31    
Test time         0.59    0.63    0.41    0.29    0.17    0.42    0.17    


{'fit_time': (10.974753618240356,
  14.03977656364441,
  14.003156661987305,
  11.78525447845459,
  7.775218486785889),
 'test_mae': array([0.73744879, 0.73014263, 0.74346226, 0.73577718, 0.74120212]),
 'test_rmse': array([0.9371979 , 0.92898809, 0.93899746, 0.93736929, 0.94161275]),
 'test_time': (0.5947346687316895,
  0.6294488906860352,
  0.40744757652282715,
  0.2943115234375,
  0.1723332405090332)}

* NMF를 사용한 협업 필터링

In [41]:
model=NMF()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9580  0.9659  0.9698  0.9684  0.9558  0.9636  0.0056  
MAE (testset)     0.7533  0.7583  0.7591  0.7627  0.7525  0.7572  0.0038  
Fit time          14.72   17.77   17.67   14.74   9.81    14.94   2.89    
Test time         0.66    0.50    0.40    0.26    0.16    0.39    0.18    


{'fit_time': (14.723661422729492,
  17.771509170532227,
  17.66822624206543,
  14.740811347961426,
  9.811245203018188),
 'test_mae': array([0.75328127, 0.75825159, 0.75913527, 0.76271247, 0.75252029]),
 'test_rmse': array([0.95803697, 0.96585113, 0.96980299, 0.96843816, 0.95581697]),
 'test_time': (0.6575400829315186,
  0.5045182704925537,
  0.3978579044342041,
  0.25542140007019043,
  0.15656280517578125)}

* SVD++를 사용한 협업 필터링

In [42]:
model=SVDpp()
cross_validate(model,data,measures=['rmse','mae'],cv=5,n_jobs=4,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9178  0.9237  0.9150  0.9234  0.9117  0.9183  0.0047  
MAE (testset)     0.7197  0.7267  0.7170  0.7227  0.7172  0.7207  0.0037  
Fit time          650.84  646.45  653.46  649.57  206.94  561.45  177.27  
Test time         11.90   11.55   9.68    9.59    3.11    9.17    3.17    


{'fit_time': (650.8376843929291,
  646.4455416202545,
  653.4601876735687,
  649.5744187831879,
  206.94036221504211),
 'test_mae': array([0.71971226, 0.72674383, 0.71701363, 0.72270339, 0.71721192]),
 'test_rmse': array([0.91779544, 0.92372553, 0.91500801, 0.92339714, 0.91165884]),
 'test_time': (11.900418519973755,
  11.554717063903809,
  9.67935824394226,
  9.586155414581299,
  3.1052162647247314)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [67]:
import numpy as np
from sklearn.decomposition import randomized_svd,non_negative_factorization
from surprise import Dataset

In [68]:
data=Dataset.load_builtin('ml-100k',prompt=False)
raw_data=np.array(data.raw_ratings,dtype=int)
raw_data[:,0]-=1
raw_data[:,1]-=1

In [69]:
n_users=np.max(raw_data[:,0])
n_movies=np.max(raw_data[:,1])
shape=(n_users+1,n_movies+1)
shape

(943, 1682)

In [70]:
adj_matrix=np.ndarray(shape,dtype=int)
for user_id,movie_id,rating,time in raw_data:
  adj_matrix[user_id][movie_id]=rating
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

U =user, V= item ,S= tri value vector

In [71]:
U,S,V=randomized_svd(adj_matrix,n_components=2)
S=np.diag(S)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [72]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [73]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [74]:
my_id,my_vector=0,U[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(U):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999942295956324, Best Match ID : 235


In [75]:
recommend_list=[]
for i, log in enumerate(zip(adj_matrix[my_id],adj_matrix[best_match_id])):
  log1,log2=log
  if log1<1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [76]:
my_id,my_vector=0,V.T[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(V.T):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999999951364144, Best Match ID : 1287


In [77]:
recommend_list=[]
for i, log in enumerate(adj_matrix):
  if adj_matrix[i][my_id]>0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [78]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [79]:
A,B,iter=non_negative_factorization(adj_matrix,n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [80]:
np.matmul(A,B)

array([[3.71108858e+00, 1.48454102e+00, 7.39535363e-01, ...,
        3.64490531e-03, 1.45506510e-02, 1.44110916e-02],
       [2.11724416e+00, 2.37338725e-01, 5.51650703e-01, ...,
        4.76092332e-03, 3.03585167e-05, 0.00000000e+00],
       [9.85272360e-01, 1.10447010e-01, 2.56713988e-01, ...,
        2.21552443e-03, 1.41275192e-05, 0.00000000e+00],
       ...,
       [1.04485667e+00, 1.17126289e-01, 2.72238757e-01, ...,
        2.34950819e-03, 1.49818804e-05, 0.00000000e+00],
       [1.45758857e+00, 5.42112787e-01, 2.99181695e-01, ...,
        1.61177659e-03, 5.15921394e-03, 5.10771005e-03],
       [2.44733103e+00, 9.41208708e-01, 4.95740038e-01, ...,
        2.56992125e-03, 9.08287324e-03, 8.99389282e-03]])

* 사용자 기반 추천

In [81]:
my_id,my_vector=0,U[0] #suppose that My ID is the 0
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(U):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999942295956324, Best Match ID : 235


In [82]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천

In [83]:
my_id,my_vector=0,V.T[0]
best_match,best_match_id,best_match_vector=-1,-1,[]

for user_id,user_vector in enumerate(V.T):
  if my_id !=user_id:
    cos_similarity=compute_cos_similarity(my_vector,user_vector)
    if cos_similarity > best_match:
      best_match=cos_similarity
      best_match_id=user_id
      best_match_vector=user_vector

  
print('best Match: {}, Best Match ID : {}'.format(best_match,best_match_id))

best Match: 0.9999999951364144, Best Match ID : 1287


In [84]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 